In [14]:
import sys
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
from scipy import stats
from scipy.stats import rankdata
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics, preprocessing
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
np.random.seed(1)

In [15]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation',\
            'relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']
df_train = pd.read_csv('adult.data', names=cols, sep=",")
df_test = pd.read_csv('adult.test', names=cols, sep=",")

**Pre-processing** (categorical to numerical)

In [16]:
def preprocess(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['workclass'] = df['workclass'].map({'Never-worked': 0, 'Without-pay': 1, 'State-gov': 2, 'Local-gov': 3, 'Federal-gov': 4, 'Self-emp-inc': 5, 'Self-emp-not-inc': 6, 'Private': 7}).astype(int)
    df['education'] = df['education'].map({'Preschool': 0, '1st-4th': 1, '5th-6th': 2, '7th-8th': 3, '9th': 4, '10th': 5, '11th': 6, '12th': 7, 'HS-grad':8, 'Some-college': 9, 'Bachelors': 10, 'Prof-school': 11, 'Assoc-acdm': 12, 'Assoc-voc': 13, 'Masters': 14, 'Doctorate': 15}).astype(int)
    df['marital'] = df['marital'].map({'Married-civ-spouse': 2, 'Divorced': 1, 'Never-married': 0, 'Separated': 1, 'Widowed': 1, 'Married-spouse-absent': 2, 'Married-AF-spouse': 2}).astype(int)
    df['relationship'] = df['relationship'].map({'Wife': 1 , 'Own-child': 0 , 'Husband': 1, 'Not-in-family': 0, 'Other-relative': 0, 'Unmarried': 0}).astype(int)
    df['race'] = df['race'].map({'White': 1, 'Asian-Pac-Islander': 0, 'Amer-Indian-Eskimo': 0, 'Other': 0, 'Black': 0}).astype(int)
    df['gender'] = df['gender'].map({'Male': 1, 'Female': 0}).astype(int)
    
    # process hours
    df.loc[(df['hours'] <= 40), 'hours'] = 0
    df.loc[(df['hours'] > 40), 'hours'] = 1

    df = df.drop(columns=['fnlwgt', 'education.num', 'occupation', 'country', 'capgain', 'caploss'])
    df = df.reset_index(drop=True)
    return df

df_train = preprocess(df_train)
df_test = preprocess(df_test)

X_train = copy.deepcopy(df_train)
X_train = X_train.drop(columns=['income'])
y_train = df_train['income']

X_test = copy.deepcopy(df_test)
X_test = X_test.drop(columns=['income'])
y_test = df_test['income']

**Parametric Model**

In [17]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=300)

**Compute fairness metric**

In [18]:
def computeFairness(y_pred, X_test, y_test, metric): 
    fairnessMetric = 0
    protected_idx = X_test[X_test['gender']==0].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['gender']==1].index
    numPrivileged = len(privileged_idx)
        
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    # statistical parity difference
    statistical_parity = p_protected - p_privileged
    
#     # equality of opportunity, or 
#     # true positive rate parity
#     # P(Y=1 | Y=1, G=0)- P(Y=1 | Y=1, G=1)
#     true_positive_protected = 0
#     actual_positive_protected = 0
#     for i in range(len(protected_idx)):
#         if (y_test[protected_idx[i]] == 1):
#             actual_positive_protected += 1
# #             if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
#             true_positive_protected += y_pred[protected_idx[i]][1]
#     tpr_protected = true_positive_protected/actual_positive_protected
    
#     true_positive_privileged = 0
#     actual_positive_privileged = 0
#     for i in range(len(privileged_idx)):
#         if (y_test[privileged_idx[i]] == 1):
#             actual_positive_privileged += 1
# #             if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
#             true_positive_privileged += y_pred[privileged_idx[i]][1]
#     tpr_privileged = true_positive_privileged/actual_positive_privileged
    
#     tpr_parity = tpr_protected - tpr_privileged
    
#     # equalized odds or TPR parity + FPR parity
#     # false positive rate parity
    
#     # predictive parity
#     p_o1_y1_s1 = 0
#     p_o1_s1 = 0
#     for i in range(len(protected_idx)):
# #         if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
#         p_o1_s1 += y_pred[protected_idx[i]][1]
#         if (y_test[protected_idx[i]] == 1):
#             p_o1_y1_s1 += y_pred[protected_idx[i]][1]
#     ppv_protected = p_o1_y1_s1/p_o1_s1
    
#     p_o1_y1_s0 = 0
#     p_o1_s0 = 0
#     for i in range(len(privileged_idx)):
# #         if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
#         p_o1_s0 += y_pred[privileged_idx[i]][1]
#         if (y_test[privileged_idx[i]] == 1):
#             p_o1_y1_s0 += y_pred[privileged_idx[i]][1]
#     ppv_privileged = p_o1_y1_s0/p_o1_s0
    
#     predictive_parity = ppv_protected - ppv_privileged
    
    if (metric == 0):
        fairnessMetric = statistical_parity
    elif (metric == 1):
        fairnessMetric = tpr_parity
    elif (metric == 2):
        fairnessMetric = predictive_parity
        
    return fairnessMetric

**Loss function** (Log loss for logistic regression)

In [19]:
def logistic_loss(y_true, y_pred):
    loss = 0
    for i in range(len(y_true)):
        if (y_pred[i][1] != 0 and y_pred[i][0] != 0):
            loss += - y_true[i] * math.log(y_pred[i][1]) - (1 - y_true[i]) * math.log(y_pred[i][0])
    loss /= len(y_true)
    return loss

**Compute Accuracy** 

In [20]:
from sklearn.metrics import accuracy_score

def computeAccuracy(y_true, y_pred):
    accuracy = 0
    for i in range(len(y_true)):
        idx = y_true[i]
        if (y_pred[i][idx] > y_pred[i][1 - idx]):
            accuracy += 1
#         accuracy += y_pred[i][idx]
    accuracy /= len(y_true)
    return accuracy

**First-order derivative of loss function at z with respect to model parameters**

In [21]:
def del_L_del_theta_i(num_params, y_true, x, y_pred):
#     del_L_del_theta = np.ones((num_params, 1)) * ((1 - y_true) * y_pred[1] - y_true * y_pred[0])
    del_L_del_theta = np.ones((num_params, 1)) * (- y_true + y_pred[1])
    for j in range(1, num_params):
            del_L_del_theta[j] *=  x[j-1]
    return del_L_del_theta

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [22]:
def del_f_del_theta_i(num_params, x, y_pred):
    del_f_del_theta = np.ones((num_params, 1)) * (y_pred[0] * y_pred[1])
    for j in range(1, num_params):
            del_f_del_theta[j] *=  x[j-1]
    return del_f_del_theta

**Computing $v=\nabla($Statistical parity difference$)$**

In [23]:
# Return v = del(SPD)/del(theta)
def del_spd_del_theta(num_params, X_test_orig, X_test, y_pred):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    numPrivileged = X_test_orig['gender'].sum()
    numProtected = len(X_test_orig) - numPrivileged
    for i in range(len(X_test)):
        del_f_i = del_f_del_theta_i(num_params, X_test[i], y_pred[i])
        if X_test_orig.iloc[i]['gender'] == 1: #privileged
            del_f_privileged = np.add(del_f_privileged, del_f_i)
        elif X_test_orig.iloc[i]['gender'] == 0:
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_privileged /= numPrivileged
    del_f_protected /= numProtected
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Computing $v=\nabla($TPR parity difference$)$**

In [24]:
# Return v = del(TPR_parity)/del(theta)
def del_tpr_parity_del_theta(num_params, X_test_orig, X_test, y_pred, y_test):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    
    protected_idx = X_test_orig[X_test_orig['gender']==0].index
    privileged_idx = X_test_orig[X_test_orig['gender']==1].index

    actual_positive_privileged = 0
    for i in range(len(privileged_idx)):
        if (y_test[privileged_idx[i]] == 1):
            actual_positive_privileged += 1
#             if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
            del_f_i = del_f_del_theta_i(num_params, X_test[privileged_idx[i]], y_pred[privileged_idx[i]])
            del_f_privileged = np.add(del_f_privileged, del_f_i)
    del_f_privileged /= actual_positive_privileged
    
    actual_positive_protected = 0
    for i in range(len(protected_idx)):
        if (y_test[protected_idx[i]] == 1):
            actual_positive_protected += 1
#             if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
            del_f_i = del_f_del_theta_i(num_params, X_test[protected_idx[i]], y_pred[protected_idx[i]])
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_protected /= actual_positive_protected

    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Computing $v=\nabla($Predictive parity difference$)$**

In [25]:
# Return v = del(Predictive_parity)/del(theta)
def del_predictive_parity_del_theta(num_params, X_test_orig, X_test, y_pred, y_test):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    
    protected_idx = X_test_orig[X_test_orig['gender']==0].index
    privileged_idx = X_test_orig[X_test_orig['gender']==1].index

    u_dash_protected = np.zeros((num_params, 1))
    v_protected = 0
    v_dash_protected = np.zeros((num_params, 1))
    u_protected = 0
    for i in range(len(protected_idx)):
        del_f_i = del_f_del_theta_i(num_params, X_test[protected_idx[i]], y_pred[protected_idx[i]])
#         if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
        v_protected += y_pred[protected_idx[i]][1]
        v_dash_protected = np.add(v_dash_protected, del_f_i)
        if (y_test[protected_idx[i]] == 1):
            u_dash_protected = np.add(u_dash_protected, del_f_i)
            u_protected += y_pred[protected_idx[i]][1]
    del_f_protected = (u_dash_protected * v_protected - u_protected * v_dash_protected)/(v_protected * v_protected)
    
    u_dash_privileged = np.zeros((num_params, 1))
    v_privileged = 0
    v_dash_privileged = np.zeros((num_params, 1))
    u_privileged = 0
    for i in range(len(privileged_idx)):
        del_f_i = del_f_del_theta_i(num_params, X_test[privileged_idx[i]], y_pred[privileged_idx[i]])
#         if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
        v_privileged += y_pred[privileged_idx[i]][1]
        v_dash_privileged = np.add(v_dash_privileged, del_f_i)
        if (y_test[privileged_idx[i]] == 1):
            u_dash_privileged = np.add(u_dash_privileged, del_f_i)
            u_privileged += y_pred[privileged_idx[i]][1]
    del_f_privileged = (u_dash_privileged * v_privileged - u_privileged * v_dash_privileged)/(v_privileged * v_privileged)
    
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Metrics: Initial state**

In [26]:
clf.fit(X_train, y_train)
num_params = len(clf.coef_.transpose()) + 1 #weights and intercept; params: clf.coef_, clf.intercept_
y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0)
print("Initial statistical parity: ", spd_0)

# tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1)
# print("Initial TPR parity: ", tpr_parity_0)

# predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2)
# print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.20054502910275857
Initial loss:  0.3954179647422569
Initial accuracy:  0.8032536520584329


**Pre-compute: del_L_del_theta for each training data point**

In [27]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    del_L_del_theta.insert(i, del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred_train[i]))

*Select delta fairness function depending on selected metric*

In [28]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(num_params, X_test_orig, X_test, y_pred_test, y_test)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(num_params, X_test_orig, X_test, y_pred_test, y_test)

** Update**

In [29]:
def del_L_del_delta_i(num_params, x, y_pred, params, y_true):
    del_L_del_delta = np.ones((num_params - 1, num_params)) * (y_pred[0] * y_pred[1])
    for i in range(num_params - 1):
        for j in range(num_params):
            del_L_del_delta[i][j] *=  params[i]
            if j != 0:
                del_L_del_delta[i][j] *=  x[j - 1]
                if j == i:
                    del_L_del_delta[i][j] += y_pred[1] - y_true
            
    return del_L_del_delta

In [30]:
def repair(idx, numIter, learningRate):
    clf.fit(X_train, y_train)
    y_pred_test = clf.predict_proba(X_test)
    
    X_p = copy.deepcopy(X_train[idx])
    y_p_true = copy.deepcopy(y_train[idx]) 

    del_L_del_delta = np.zeros((num_params - 1, num_params))

    random.seed(0) # seed random number generator
    delta_new = np.zeros((num_params - 1, 1))
    for i in range(len(delta_new)):
        delta_new[i] = random.random()
    
    threshold = 0.05
    delta_old = -1 * np.ones((num_params - 1, 1))
    v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)

    num_iter = 0
    del_L_del_theta_Xp = np.zeros((num_params, 1))
    for i in range(len(idx)):
        del_L_del_theta_Xp = np.subtract(del_L_del_theta_Xp, del_L_del_theta_i(num_params, y_train[idx[i]], X_train[idx[i]], y_pred_train[i]))
    
    obj_old = np.dot(del_L_del_theta_Xp.transpose(), v1)
    obj_new = obj_old - 0.1
    print(obj_old, obj_new)
    
#     while (np.linalg.norm(np.subtract(delta_old, delta_new)) > threshold):
#         print(np.linalg.norm(np.subtract(delta_old, delta_new)))
    while (obj_new < obj_old):
#     while True:
#     while ((num_iter < 100) or (obj_new - obj_old > threshold)) :
        obj_old = obj_new
#         print(num_iter)
        num_iter += 1
        for i in range(len(idx)):
            x = np.zeros((len(X_train[idx[i]]), 1))
            for j in range(len(x)):
                x[j] = X_p[i][j]
            x = np.add(x, delta_new)
            x_ = [x[i][0] for i in range(len(x))]
            y_pred = clf.predict_proba([x_])
            del_L_del_delta_i_ = del_L_del_delta_i(num_params, x, y_pred[0], clf.coef_[0], y_train[idx[i]])/len(idx)
            del_L_del_delta = np.add(del_L_del_delta, del_L_del_delta_i_)
        
        delta_old = delta_new
        delta_new = np.add(delta_new, (learningRate/num_iter) * np.dot(del_L_del_delta, v1))
        
        X_train_perturbed = copy.deepcopy(X_train)
        for i in range(len(idx)):
            for j in range(len(delta_new)):
                X_train_perturbed[idx[i]][j] += delta_new[j]
            
        del_L_del_theta_Xp = np.zeros((num_params, 1))
        for i in range(len(idx)):
            del_L_del_theta_Xp = np.add(del_L_del_theta_Xp, del_L_del_theta_i(num_params, y_train[idx[i]], X_train_perturbed[idx[i]], y_pred_train[idx[i]]))

        obj_new = np.dot(del_L_del_theta_Xp.transpose(), v1)
        print(obj_old, obj_new)
#         if (numIter>=10):
#         if ((num_iter > 10) & (obj_new - obj_old < threshold)):
#         if ((num_iter > 2) & (obj_new < obj_old)):
        if ((obj_new > obj_old)):
#             print((obj_new - obj_old))
            return delta_old
    
    return delta_new

In [31]:
idx = X_train_orig[
    (X_train_orig['gender'] == 0)
    & (X_train_orig['relationship'] == 0)
    & (X_train_orig['education'] == 12)
    & (X_train_orig['race'] == 1)
    ].index 
# v_pert = repair(idx, numIter, learningRate)
v_pert = repair(idx, 100, 1)

[[-24.71518334]] [[-24.81518334]]
[[-24.81518334]] [[2.26371456]]


In [35]:
df = copy.deepcopy(X_train_orig)
df['income'] = y_train
df.iloc[idx]

,age,workclass,education,marital,relationship,race,gender,hours,income
90,1,7,12,1,0,1,0,0,0
120,0,7,12,0,0,1,0,1,0
201,1,7,12,1,0,1,0,0,0
285,1,7,12,0,0,1,0,0,0
340,0,7,12,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
29779,1,6,12,0,0,1,0,0,0
30004,0,7,12,1,0,1,0,0,0
30131,0,7,12,0,0,1,0,0,0
30144,1,3,12,1,0,1,0,1,0


In [36]:
X_test_orig

,age,workclass,education,marital,relationship,race,gender,hours
0,0,7,6,0,0,0,1,0
1,0,7,8,2,1,1,1,1
2,0,3,12,2,1,1,1,0
3,0,7,9,2,1,0,1,0
4,0,7,5,0,0,1,1,0
...,...,...,...,...,...,...,...,...
15055,0,7,10,0,0,1,1,0
15056,0,7,10,1,0,1,0,0
15057,0,7,10,2,1,1,1,1
15058,0,7,10,1,0,0,1,0


In [37]:
y_pred_test

array([[0.98875209, 0.01124791],
       [0.57447972, 0.42552028],
       [0.42346111, 0.57653889],
       ...,
       [0.41920909, 0.58079091],
       [0.93570663, 0.06429337],
       [0.40372845, 0.59627155]])

In [41]:
%%time
res_age = []
res_workclass = []
res_education = []
res_marital = []
res_relationship = []
res_race = []
res_gender = []
res_hours = []

X_train_copy = copy.deepcopy(X_train)
numCols = len(X_train_orig.columns)
for ix in range(len(idx)):
    X_train_pert = np.zeros((len(X_train[idx[ix]]), 1))
    for i in range(len(X_train[idx[ix]])):
        X_train_pert[i] = X_train[idx[ix]][i] + v_pert[i]

    x0 = np.random.rand(1,numCols)
    mins = []
    maxs = []
    numCols = len(X_train[0])
    for i in range(numCols):
        mins.insert(i, min(X_train[:,i]))
        maxs.insert(i, max(X_train[:,i]))

    from scipy.optimize import Bounds, minimize
    bounds = Bounds(mins, maxs)

    f = lambda x: np.linalg.norm(x - X_train_pert)

    x0 = np.random.rand(numCols)
    res = minimize(f, x0, method='trust-constr', 
    #                jac=rosen_der, hess=rosen_hess,
    #                constraints=[linear_constraint, nonlinear_constraint],
                   options={'verbose': 0}, bounds=bounds)
    
    for i in range(len(X_train_copy[idx[ix]])):
        X_train_copy[idx[ix]][i] = res.x[i]
        
#     res_x_inv_transform = sc.inverse_transform(res.x, copy=None)
    res_x_inv_transform = sc.inverse_transform(X_train_copy[idx[ix]], copy=None)
#     res_x_inv_transform = sc.inverse_transform(X_train_copy[idx[ix]], copy=None)
    res_age.insert(ix, res_x_inv_transform[0])
    res_workclass.insert(ix, res_x_inv_transform[1])
    res_education.insert(ix, res_x_inv_transform[2])
    res_marital.insert(ix, res_x_inv_transform[3])
    res_relationship.insert(ix, res_x_inv_transform[4])
    res_race.insert(ix, res_x_inv_transform[5])
    res_gender.insert(ix, res_x_inv_transform[6])
    res_hours.insert(ix, res_x_inv_transform[7])

clf.fit(X_train_copy, y_train)
y_pred_test = clf.predict_proba(X_test)

import statistics
print(statistics.mode(res_age), statistics.mode(res_workclass), statistics.mode(res_education), 
      statistics.mode(res_marital), statistics.mode(res_relationship), 
      statistics.mode(res_race), statistics.mode(res_gender), statistics.mode(res_hours))

print(computeFairness(y_pred_test, X_test_orig, y_test, 0)
      /spd_0 - 1
     )
# print(spd_0)


/Users/dennis/Library/Python/3.8/lib/python/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


0.5857466063709567 6.999999744642305 10.402278509001901 1.6749982248419581 0.7527655239776361 0.999999985475235 0.9508438029616417 0.5755269067442623
-0.02113477675421438
CPU times: user 1min 10s, sys: 575 ms, total: 1min 10s
Wall time: 1min 13s


In [42]:
clf.coef_

array([[ 0.24747855, -0.05325963,  0.7727246 ,  0.49329915,  0.73899166,
         0.06315162,  0.0729862 ,  0.35324613]])

In [ ]:
Initial: [[ 0.32698744  2.20756884  0.52520941  0.26284814 -0.14408554]]

In [26]:
df = pd.read_table('salary.data', names=cols, sep="\t", index_col=False)
df = preprocess(df)
# df[df['status', 'credit'].groupby(by="credit").sum()

In [39]:
pd.DataFrame(X_train_orig.columns)

,0
0,age
1,workclass
2,education
3,marital
4,relationship
5,race
6,gender
7,hours
